In [ ]:
#instalaciones requeridas
#pip install pandas
#pip install gradio
#pip install gensim
#pip install "numpy<2"
#pip install sklearn
#pip install nltk

In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
import gradio as gr
import nltk

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Preproceso y entrenamiento del modelo de embeddings
def embed_text(path="texto.csv"):
    # Cargar los datos del archivo CSV
    conocimiento_df = pd.read_csv(path)
    
    # Tokenizar cada texto
    conocimiento_df['tokens'] = conocimiento_df['texto'].apply(word_tokenize)
    
    # Entrenar Word2Vec en los tokens
    model = Word2Vec(conocimiento_df['tokens'], vector_size=100, window=5, min_count=1, workers=4)
    
    # Obtener los embeddings de cada documento (promedio de los embeddings de las palabras)
    conocimiento_df['Embedding'] = conocimiento_df['tokens'].apply(lambda tokens: np.mean([model.wv[word] for word in tokens if word in model.wv], axis=0))
    
    # Guardar los embeddings en un archivo CSV
    conocimiento_df.to_csv('embeddings.csv', index=False)
    
    return conocimiento_df, model

In [4]:
# Búsqueda con similitud de coseno
def buscar(busqueda, datos, model, n_resultados=5):
    # Tokenizar la búsqueda
    tokens_busqueda = word_tokenize(busqueda)
    
    # Cargar el modelo Word2Vec (puedes cargar el mismo modelo entrenado anteriormente)
    #model = Word2Vec([tokens_busqueda], vector_size=100, window=5, min_count=1, workers=4)
    model.build_vocab([tokens_busqueda], update=True)  # Actualiza el vocabulario del modelo
    model.train([tokens_busqueda], total_examples=1, epochs=1)  # Reentrena el modelo con la nueva búsqueda
    
    # Generar embedding para la búsqueda (promedio de los embeddings de las palabras)
    busqueda_embed = np.mean([model.wv[word] for word in tokens_busqueda if word in model.wv], axis=0).reshape(1, -1)
    
    # Calcular la similitud de coseno entre la búsqueda y cada documento
    datos['Similitud'] = datos['Embedding'].apply(lambda x: cosine_similarity(np.array(x).reshape(1, -1), busqueda_embed)[0][0])
    
    # Ordenar por similitud y devolver los resultados más relevantes
    datos = datos.sort_values("Similitud", ascending=False)
    return datos.iloc[:n_resultados][["texto", "Similitud", "Embedding"]]


In [5]:
# Embedding del texto
texto_emb, model = embed_text("./chatbot_qa.csv")


In [6]:
def buscar_wrapper(busqueda, texto_emb):
    return buscar(busqueda, texto_emb, model)

# Gradio UI
with gr.Blocks() as demo:
    busqueda = gr.Textbox(label="Buscar")
    output = gr.DataFrame(headers=['texto'])
    greet_btn = gr.Button("Preguntar")
    
    greet_btn.click(fn=buscar_wrapper, inputs=[busqueda, gr.DataFrame(texto_emb)], outputs=output)

demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [8]:
resultado=buscar_wrapper("estan abiertos", texto_emb)

In [15]:
resultado

,texto,Similitud,Embedding
1,Estamos abiertos de Lunes a Sabado de 9 a 16:00,0.145068,"[-0.0007152733, 3.1758846e-05, 0.002224551, -0..."
5,Las cotizaciones se hacen una vez que ya se de...,0.078098,"[-0.00069736206, -0.0010509983, -0.0007322012,..."
0,Cerramos los domingos,-0.003760,"[0.0005585933, 0.0050890553, -0.00076500064, 0..."
3,Contamos con servicios de carpinteria y herrería,-0.017286,"[-0.0036363613, 0.00035885003, -0.0033940468, ..."
4,No contamos con servicio de jardinería,-0.045271,"[0.0008710359, 0.00022980718, 0.0008115939, -0..."


# Ejercicio mas rudimentario

# Word2Vec

In [19]:
import string
from gensim.models import Word2Vec

In [21]:
with open('Minecraft.txt', 'r', encoding='utf-8') as file:
    documento = file.read()

In [23]:
# Dividir el documento en frases usando el punto como separador
oraciones = documento.split(',')
len(oraciones)

198

In [25]:
oraciones[0]

'Minecraft \nMinecraft es un videojuego de construcción de tipo «mundo abierto» o sandbox creado originalmente por el sueco Markus Persson (conocido comúnmente como «Notch»)'

In [27]:
oraciones_limpias = []
for oracion in oraciones:
    # Eliminar puntuación y dividir por espacios
    tokens = oracion.translate(str.maketrans('', '', 
                                        string.punctuation)).split()
    # Convertir a minúsculas
    tokens = [word.lower() for word in tokens if word.isalpha()]
    if tokens:  # Añadir solo si hay tokens
        oraciones_limpias.append(tokens)

In [29]:
oraciones_limpias[0]

['minecraft',
 'minecraft',
 'es',
 'un',
 'videojuego',
 'de',
 'construcción',
 'de',
 'tipo',
 'o',
 'sandbox',
 'creado',
 'originalmente',
 'por',
 'el',
 'sueco',
 'markus',
 'persson',
 'conocido',
 'comúnmente',
 'como']

## Entrenamiento del modelo Word2Vec 

In [32]:
# Entrenar el modelo Word2Vec
model = Word2Vec(sentences=oraciones_limpias, 
                 vector_size=500, window=5, min_count=1, workers=8)

In [34]:
vector = model.wv['minecraft']

In [36]:
vector

array([ 1.31467485e-03,  1.72488671e-03,  1.61889032e-03,  2.37747072e-03,
       -1.62376475e-03, -1.44111132e-03,  2.11569527e-03,  2.77249445e-03,
        1.01266417e-03,  8.05176503e-04, -2.19122827e-04, -1.27422111e-03,
        1.04291993e-03, -1.30581344e-03,  6.94971022e-05, -3.00907250e-03,
       -1.93236396e-03, -2.70003034e-03,  1.20208610e-03, -1.34947873e-03,
        2.30061612e-03, -2.35353736e-03, -9.58908859e-05,  6.28849550e-04,
       -3.08468647e-04,  8.10732075e-04, -4.96394758e-04, -6.10958887e-05,
       -3.18940403e-03, -2.11702893e-03,  1.65419409e-03,  3.93253198e-04,
        1.74455554e-03, -1.67894014e-03,  1.72165618e-03, -1.03814702e-03,
        2.13829940e-03,  4.13459900e-04, -1.98268401e-03,  1.99841787e-04,
       -1.86780014e-03, -2.27080146e-03, -1.21268374e-03,  1.01278524e-03,
       -3.19198146e-03, -2.88477447e-03,  8.34073056e-04,  1.24955608e-03,
       -1.92226827e-04,  3.36185010e-04, -9.26160137e-04, -1.92146195e-04,
       -1.97093599e-04,  

In [38]:
palabras_cercanas = model.wv.most_similar("minecraft", topn=10)
palabras_cercanas

[('en', 0.5539902448654175),
 ('de', 0.5418345928192139),
 ('la', 0.46545806527137756),
 ('el', 0.4584834575653076),
 ('jugadores', 0.45203712582588196),
 ('los', 0.4388474225997925),
 ('que', 0.4329295754432678),
 ('para', 0.4269357919692993),
 ('y', 0.422042578458786),
 ('a', 0.41557562351226807)]